In [ ]:
from sympy import symbols, cos, sin, diff, Function, simplify

# Define symbols
theta = symbols('theta')
k, e, h, GM, sigma = symbols('k e h GM sigma')

# Define r(theta)
r = (h ** 2 / GM) / (1 +  e * cos(theta * sigma))

# Calculate dr/dtheta
dr_dtheta = diff(r, theta)

# Define omega(theta) = dtheta/dt
omega = h / r**2

# Calculate d^2r/dtheta^2
d2r_dtheta2 = diff(dr_dtheta, theta)

# Calculate domega/dtheta
domega_dtheta = diff(omega, theta)

dr_dt = diff(r,theta)*omega
# Calculate d^2r/dt^2 using chain rule
d2r_dt2 = simplify(d2r_dtheta2 * omega**2 + dr_dtheta * domega_dtheta * omega)

# Calculate d^2theta/dt^2
# Differentiate omega(theta)
d2theta_dt2 = simplify(domega_dtheta * omega)

# Display the results

eq1 = f"r= {r}".replace("cos", "np.cos").replace("sin", "np.sin")
eq2 = f"theta_dot= h/r**2".replace("cos", "np.cos").replace("sin", "np.sin")
eq3 = f"dr_dtheta = {dr_dtheta}".replace("cos", "np.cos").replace("sin", "np.sin")
eq4 = f"d2r_dtheta2 = {d2r_dtheta2}".replace("cos", "np.cos").replace("sin", "np.sin")
eq5 = f"domega_dtheta = {domega_dtheta}".replace("cos", "np.cos").replace("sin", "np.sin")
eq6 = f"d2r_dt2 = {d2r_dt2}".replace("cos", "np.cos").replace("sin", "np.sin")
eq7 = f"d2theta_dt2 = {d2theta_dt2}".replace("cos", "np.cos").replace("sin", "np.sin")
eq8 = f"dr_dt = {dr_dt}".replace("cos", "np.cos").replace("sin", "np.sin")

print(eq1)
print(eq2)
print(eq3)
print(eq4)
print(eq5)
print(eq6)
print(eq7)
print(eq8)

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu

from math import pi

# Constants
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
M_sun = 1.989e30  # Mass of the Sun in kg
c = 299792458  # Speed of light in m/s
GM = G * M_sun


def r_derivatives(theta, h, e, GM, sigma):
    # Expression for r, r_dot, and r_double_dot
    r = h**2 / (GM * (e * np.cos(sigma * theta) + 1))
    theta_dot = h / r**2
    dr_dtheta = e * h**2 * sigma * np.sin(sigma * theta) / (GM * (e * np.cos(sigma * theta) + 1)**2)
    d2r_dtheta2 = 2 * e**2 * h**2 * sigma**2 * np.sin(sigma * theta)**2 / (GM * (e * np.cos(sigma * theta) + 1)**3) + e * h**2 * sigma**2 * np.cos(sigma * theta) / (GM * (e * np.cos(sigma * theta) + 1)**2)
    d2r_dt2 = GM**3 * e * sigma**2 * (e * np.cos(sigma * theta) + 1)**2 * np.cos(sigma * theta) / h**4
    dr_dt = GM * e * sigma * np.sin(sigma * theta) / h
    return r, dr_dt, d2r_dt2, theta_dot


def error_functionHU(x, h, e, GM):
    def integrand(theta):
        r, r_dot, r_double_dot, theta_dot = r_derivatives(theta, h, e, GM, x[0])
        a_theoretical = accel_HU(GM, r, r_dot, theta_dot)
        a_numerical = r_double_dot - r * theta_dot**2
        return (a_theoretical - a_numerical)**2

    sigma = x[0]
    integral_error, _ = quad(integrand, 0, 2 * np.pi, epsabs=1e-14, epsrel=1e-14)
    return 100 * integral_error


def accel_HU(GM, r, r_dot, theta_dot):
    v = np.sqrt(r_dot**2 + r**2 * theta_dot**2)
    gamma_v = 1 / np.sqrt(1 - v**2 / c**2)
    A = gamma_v**2
    a_theoretical = -GM / r**2 / A
    return a_theoretical


def calculatePrecession(errorf, a, e, T):
    h = np.sqrt(GM * a * (1 - e**2))
    # Initial guess for sigma
    initial_x = [0.95]
    result = minimize(errorf, initial_x, args=(h, e, GM), method='Nelder-Mead', tol=1E-16)
    optimized_sigma = result.x[0]  # The optimized value for sigma
    print(result.x)
    days_in_century = 36525  # Number of days in a century

    # Calculation of precession per orbit in radians
    delta_phi_per_orbit = 2 * np.pi * (optimized_sigma - 1)
    delta_phi_per_orbit_GR = (6 * pi * GM) / (a * (1 - e**2) * c**2)

    # Calculate the number of orbits per century
    orbits_per_century = days_in_century / T

    # Precession per century in arcseconds
    delta_phi_per_century = delta_phi_per_orbit * orbits_per_century * (180 / pi) * 3600  # Convert radians to arcseconds
    delta_phi_per_century_GR = delta_phi_per_orbit_GR * orbits_per_century * (180 / pi) * 3600  # Convert radians to arcseconds

    return delta_phi_per_century, delta_phi_per_century_GR


a_ = [57.91E9, 108.2E9, 149.6E9, 227.9E9, 778.5E9, 1433.5E9, 2872.5E9, 4495.1E9]
e_ = [0.205630, 0.0067, 0.0167, 0.0934, 0.0489, 0.0565, 0.0457, 0.0113]
T_ = [88, 224.7, 365.25, 687, 4333, 10759, 30660, 60182]

planet = ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]

df = pd.DataFrame(index=planet, columns=['planet', 'delta_HU', "delta_GR"])
df["planet"] = planet
for i, (a, e, T, name) in enumerate(zip(a_, e_, T_, planet)):
    df.loc[name, ['delta_HU', "delta_GR"]] = calculatePrecession(error_functionHU, a, e, T)

ax = df.plot.scatter(x="planet", y='delta_HU')
df.plot(x="planet", y='delta_GR', ax=ax)

plt.show()


In [ ]:
df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from astropy import constants as cc, units as uu

# Constants
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
M_sun = 1.989e30  # Mass of the Sun in kg
r_sun = cc.R_sun.si.value
GM = G * M_sun
c = 299792458  # Speed of light in m/s

# Semi-major axis and eccentricity for Mercury
a_mercury = 57.91e9  # meters
e_mercury = 0.0 #0.205630
T_mercury = 88 * 24 * 3600  # Orbital period in seconds

# Initial conditions
r_0 = a_mercury * (1 - e_mercury)  # Perihelion distance
theta_0 = 0  # Starting at perihelion
theta_dot_0 =  np.sqrt(GM * a_mercury * (1 + e_mercury) )/ r_0**2
r_dot_0 = 0  # Starting at perihelion, radial speed is 0
h = r_0**2*theta_dot_0
# Velocity dependent acceleration function
def accel_HU(t, y):
    r, theta, r_dot, theta_dot = y
    v = np.sqrt(r_dot**2 + (r*theta_dot)**2)
#     print(theta, r_dot)
    if v>c:
        print("FTL")
    gamma_v = 1 / np.sqrt(1 - v**2 / c**2)
    A = (1 + v**2 / c**2 - 2 * r_dot / c)**(3 / 2)
    B = (1 + (gamma_v - 1) * (r_dot / v)**2)
    a_r = -GM / r**2 # / gamma_v / B / A
    a_theta = 0.0 # 2* h/r**3 *  r_dot
    return [r_dot, theta_dot, a_r, a_theta]

# Integration time span
total_time = 100* 365.25 * 24 * 3600  # 100 years in seconds

# Integrate the equations of motion
sol = solve_ivp(accel_HU, [0, T_mercury], [r_0, theta_0, r_dot_0, theta_dot_0 ],  method='RK23', rtol=1e-16, atol=1e-16)

# Analyzing the results for precession
r, theta = sol.y[0], sol.y[1]
theta_final = theta[-1]
sols_in_century = total_time/T_mercury

print("final",  sols_in_century, theta_final)
# Calculate the precession per century
precession_radians = theta_final - 2 * np.pi
precession_arcseconds =total_time/T_mercury * precession_radians * 360 * 3600 / (2 * np.pi)

# Plotting the results
# plt.polar(np.degrees(theta), r)  # This will plot in polar coordinates and wrap theta
plt.polar(theta, r)  # This will plot in polar coordinates and wrap theta
# plt.xlabel('Theta (radians)')
# plt.ylabel('Radius (m)')
plt.title('Orbit of Mercury')
plt.show()

print(f"Precession of Mercury's orbit in arcseconds per century: {precession_arcseconds}")


In [ ]:
sol

In [ ]:
plt.plot(theta)

In [ ]:
from math import pi

# Constants
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
M_sun = 1.989e30  # Mass of the Sun in kg
c = 299792458  # Speed of light in m/s

# Venus's orbital parameters
a_venus = 108.2e9  # Semi-major axis in meters
e_venus = 0.67  # Eccentricity
orbital_period_venus_days = 224.7  # Orbital period in Earth days
days_in_century = 36525  # Number of days in a century

# Calculation of precession per orbit in radians
delta_phi_per_orbit = (6 * pi * G * M_sun) / (a_venus * (1 - e_venus**2) * c**2)

# Calculate the number of orbits per century
orbits_per_century = days_in_century / orbital_period_venus_days

# Precession per century in arcseconds
delta_phi_per_century = delta_phi_per_orbit * orbits_per_century * (180/pi) * 3600  # Convert radians to arcseconds

delta_phi_per_century


In [ ]:
# Mercury's orbital parameters
a_mercury = 57.91e9  # Semi-major axis in meters
e_mercury = 0.205630  # Eccentricity
orbital_period_mercury_days = 88  # Orbital period in Earth days

# Calculation of precession per orbit for Mercury in radians
delta_phi_per_orbit_mercury = (6 * pi * G * M_sun) / (a_mercury * (1 - e_mercury**2) * c**2)

# Calculate the number of orbits per century for Mercury
orbits_per_century_mercury = days_in_century / orbital_period_mercury_days

# Precession per century for Mercury in arcseconds
delta_phi_per_century_mercury = delta_phi_per_orbit_mercury * orbits_per_century_mercury * (180/pi) * 3600  # Convert radians to arcseconds

delta_phi_per_century_mercury


In [ ]:
# Re-declaring constants and parameters
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
M_sun = 1.989e30  # Mass of the Sun in kg

# Orbital parameters for Mercury and Venus
a_mercury = 57.91e9  # Semi-major axis of Mercury in meters
a_venus = 108.2e9  # Semi-major axis of Venus in meters

# Calculate average orbital speeds for Mercury and Venus using the simplified formula
v_mercury = (G * M_sun / a_mercury)**0.5
v_venus = (G * M_sun / a_venus)**0.5

v_mercury, v_venus


In [ ]:
# Recalculating the initial conditions
r_0 = a_mercury * (1 - e_mercury)  # Perihelion distance
theta_0 = 0  # Starting at perihelion
theta_dot_0 = np.sqrt(GM / r_0**3)  # Initial angular velocity

# Set up the differential equation to include the debug prints as in the user's output
def debug_accel_HU(t, y, GM, c):
    r, theta, r_dot, theta_dot = y
    v = np.sqrt(r_dot**2 + (r * theta_dot)**2)  # Total velocity
    a_r = -GM / r**2  # Radial acceleration

    # Print intermediate results
    print(t, r, v/c, r_dot/c, r * theta_dot/c)

    return [r_dot, theta_dot, a_r, 0]

# Integrate and print intermediate steps
integrate_result = solve_ivp(
    lambda t, y: debug_accel_HU(t, y, GM, c),
    [0, 1],  # Integrate from t=0 to t=1 second
    [r_0, theta_0, r_dot_0, theta_dot_0],  # Initial conditions
    method='RK45',  # Runge-Kutta method
    rtol=1e-6,  # Relative tolerance
    atol=1e-9,  # Absolute tolerance
    dense_output=True  # Generate a continuous solution
)

# The solve_ivp doesn't print by itself, we need to call the function with intermediate steps
# To replicate the steps, we need the specific times which were in the user's output
times_to_check = [0.0, 0.00015940896412470693, 0.0025219615040992147, 0.0037829422561488214]
for t in times_to_check:
    debug_accel_HU(t, integrate_result.sol(t), GM, c)

# Since the user's code prints directly in the function, there's no output here to show.
# The print statements inside the function would print directly to the console during execution.


In [ ]:
import matplotlib.pyplot as plt
import math


earth_eccentricity = 0.017

a = 1 # astronomical unit
b = (1 - earth_eccentricity**2)**0.5 # astronomical unit
t = 0 # year

x_track = []
y_track = []

while t <= 1:
    x = a * math.cos(2*math.pi * t)
    y = b * math.sin(2*math.pi * t)
    
    t += 0.001
    
    x_track.append(x)
    y_track.append(y)
    
plt.plot(x_track, y_track)
plt.show()


In [ ]:
import time
import matplotlib.pyplot as plt
x=list()
y=list()
x_in=0.5
y_in=0.0
x.append(x_in)
y.append(y_in)

class Planet:
    def __init__(self,m,rx,ry,vx,vy,G=1):
        self.m=m
        self.rx=rx
        self.ry=ry
        self.a=0
        self.ax=0
        self.ay=0
        self.vx=vx
        self.vy=vy
        self.r=(rx**2+ry**2)**(1/2)
        self.f=0
        self.G=1
        print(self.r)
    def calculateOrbit(self,dt,planet):
        self.rx=self.rx+self.vx*dt
        self.vx=self.vx+self.ax*dt
        self.ax=0
        for i in planet:
            r=((((self.rx-i.rx)**2)+((self.ry-i.ry)**2))**(1/2))
            self.ax+=-self.G*i.m*self.rx/(r**3)

        self.ry=self.ry+self.vy*dt
        self.vy=self.vy+self.ay*dt
        self.ay=0
        for i in planet:
                self.ay+=-self.G*i.m*self.ry/(r**3)
        global x,y
        x.append(self.rx)
        y.append(self.ry)

        #self.showOrbit()
    def showOrbit(self):
        print(""" X: {} Y: {} Ax: {} Ay: {}, Vx: {}, Vy: {}""".format(self.rx,self.ry,self.ax,self.ay,self.vx,self.vy))

planets=list();
earth = Planet(1,x_in,y_in,0,1.630)
sun =   Planet(1,0,0,0,0)
planets.append(sun)
for i in range(0, 100000):
    earth.calculateOrbit(0.001, planets)

plt.plot(x,y)
plt.grid()
plt.xlim(-2.0,2.0)
plt.ylim(-2.0,2.0)
plt.show()

In [ ]:
for i in range(0, 10000):
    earth.calculateOrbit(0.01, planets)